### 第1セル

In [1]:
TEXT_PATH ="大谷翔平_raw.txt"  # テキストデータのパス

### 第2セル

In [2]:
LoRA_ADAPTER_PATH = "./checkpoint-160"  # ファインチューニングフォルダのパス

### 第3セル

In [ ]:
import os
from pathlib import Path
from typing import List, Tuple

import numpy as np
import torch
from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoModelForCausalLM,
    AutoModelForSequenceClassification,
)

from rank_bm25 import BM25Okapi
try:
    from janome.tokenizer import Tokenizer as JanomeTokenizer
    _HAS_JANOME = True
except Exception:
    _HAS_JANOME = False

import re
import unicodedata
from dataclasses import dataclass
import faiss
import pykakasi
import socket

import gradio as gr

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# =========================
# RAG_CONFIG
# =========================
@dataclass
class RAG_CONFIG:
    # 入力データ
    text_path: str = "カブトムシ_raw.txt"  # 同じディレクトリ内のファイル名

    # 生成モード: "RAG" or "FT"
    gen_mode: str = "RAG"

    # チャンク関連
    chunk_mode: str = "char"       # "char" or "sentence"
    char_chunk_size: int = 100
    overlap: bool = False          # False: オーバーラップなし
    overlap_chars: int = 20        # charモード用の重なり文字数

    # 検索/リランク
    search_mode: str = "vector"    # "vector" or "keyword"
    top_k_retrieve: int = 5        # 検索段階の候補数
    top_k_final: int = 3           # LLMへ渡す件数
    rerank_mode: bool = False      # False: リランクなし / True: リランク 5→3

    # モデル（生成）
    llm_model_name: str = "./sbintuitions/sarashina2.2-3b-instruct-v0.1"
    lora_adapter: str = ""  # LoRAアダプターのパス（例: "./my-lora-adapter"）

    # モデル（検索/リランク）
    embedding_model_name: str = "./bge-m3"
    rerank_model_name: str = "./japanese-bge-reranker-v2-m3-v1"

    # 生成挙動
    use_chat_template: bool = False
    temperature: float = 0.7

    # 回答トークン制限（途中カット防止の丸め込みあり）
    answer_token_limit: int = 256
    answer_headroom: int = 64

    # 実行環境
    device: torch.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # ★ プロンプト外出し（空ならデフォルト使用）
    rag_user_prompt_template: str = ""   # placeholders: {question}, {context}
    rag_system_prompt: str = ""          # system 用（空ならデフォルト）
    ft_user_prompt_template: str = ""    # placeholders: {question}
    ft_system_prompt: str = ""           # system 用（空ならデフォルト）

    # LoRA設定
    lora_r: int = 16
    lora_alpha: int = 32
    lora_dropout: float = 0.1
    lora_target_modules: List[str] = None  # Noneなら自動検出


# =========================
# グローバル変数でモデルを保持（ChangeLLM.pyと同じ方式）
# =========================
# デフォルトLoRAアダプターパスの設定
DEFAULT_LORA_ADAPTER_PATH = "./output/checkpoint-160"  # LoRAアダプターのフォルダ名を指定（例: "./output/checkpoint-160" または "./my-lora-adapter"）

# 生成モデル関連
llm_tok = None
llm = None  # ベースモデル（常に保持）
llm_with_lora = None  # LoRA適用済みモデル（一度だけ作成）

# 埋め込みモデル関連
emb_tok = None
emb_model = None

# リランクモデル関連
rerank_tok = None
rerank_model = None

# モデルタイプ : original or finetuning
model_type = "original"

# FAISSインデックス管理（共通化）
# FAISSインデックスフォルダの設定
FAISS_INDEX_DIR = "faiss_index"

# FAISSインデックスフォルダが存在しない場合は作成
if not os.path.exists(FAISS_INDEX_DIR):
    os.makedirs(FAISS_INDEX_DIR)

# FAISSインデックスとチャンクデータを保持
faiss_index = None
chunk_data = None
last_data_file = None  # 最後に読み込んだデータファイルを記録

def find_available_port(start_port=7860, max_attempts=100):
    """利用可能なポートを見つける"""
    for port in range(start_port, start_port + max_attempts):
        try:
            with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
                s.bind(('0.0.0.0', port))
                return port
        except OSError:
            continue
    return None

def convert_to_romaji(filename):
    """日本語ファイル名をローマ字に変換"""
    try:
        # pykakasiの新しいAPIを使用
        kakasi = pykakasi.kakasi()
        
        # 拡張子を除去
        name_without_ext = Path(filename).stem
        
        # ローマ字変換
        romaji_result = kakasi.convert(name_without_ext)
        
        # pykakasiの結果がリストの場合は、hepburn形式の文字列を結合
        if isinstance(romaji_result, list):
            romaji_name = ""
            for item in romaji_result:
                if isinstance(item, dict) and 'hepburn' in item:
                    romaji_name += item['hepburn']
                elif isinstance(item, str):
                    romaji_name += item
        else:
            romaji_name = str(romaji_result)
        
        # 英数字以外の文字を除去し、スペースをアンダースコアに変換
        safe_name = re.sub(r'[^a-zA-Z0-9]', '_', romaji_name)
        safe_name = re.sub(r'_+', '_', safe_name)  # 連続するアンダースコアを1つに
        safe_name = safe_name.strip('_')  # 前後のアンダースコアを除去
        
        return safe_name if safe_name else "data"
        
    except Exception as e:
        return "data"

def get_faiss_index_path(data_filename):
    """FAISSインデックスファイルのパスを取得"""
    # 日本語判定（ひらがな、カタカナ、漢字が含まれているか）
    japanese_pattern = re.compile(r'[\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FAF]')
    
    if japanese_pattern.search(data_filename):
        # 日本語の場合、ローマ字変換
        romaji_name = convert_to_romaji(data_filename)
        return os.path.join(FAISS_INDEX_DIR, f"{romaji_name}.faiss")
    else:
        # 日本語でない場合、そのまま使用
        safe_name = re.sub(r'[^a-zA-Z0-9._-]', '_', data_filename)
        return os.path.join(FAISS_INDEX_DIR, f"{safe_name}.faiss")

def update_rag_data(cfg: RAG_CONFIG):
    """RAGデータを更新（FAISSインデックスファイルによる条件分岐）"""
    global faiss_index, chunk_data, last_data_file
    
    # ファイルから読み込み
    current_file = cfg.text_path
    try:
        text = Path(current_file).read_text(encoding="utf-8")
    except Exception as e:
        print(f"ファイル読み込みエラー: {e}")
        return False
    
    # ファイル名を取得
    current_filename = Path(current_file).name  # ファイル名（拡張子含む）
    
    # FAISSインデックスファイルのパスを取得
    faiss_index_path = get_faiss_index_path(current_filename)
    
    # 既存のインデックスファイルが存在する場合
    if os.path.exists(faiss_index_path):
        try:
            # インデックスファイルから読み込み
            faiss_index = faiss.read_index(faiss_index_path)
            # チャンクデータも再構築
            chunk_data = build_chunks(cfg, text)
            last_data_file = current_file
            print(f"既存のFAISSインデックスを読み込みました: {faiss_index_path}")
            return True
        except Exception as e:
            print(f"既存インデックスの読み込みに失敗: {e}")
            pass
    
    # 新しいデータを読み込み、インデックスを構築
    if current_file != last_data_file or faiss_index is None:
        try:
            chunk_data = build_chunks(cfg, text)
            
            # ベクトル検索モードの場合のみFAISSインデックスを構築
            if cfg.search_mode == "vector":
                # 埋め込みモデルを読み込み
                emb_tok = AutoTokenizer.from_pretrained(cfg.embedding_model_name)
                emb_model = AutoModel.from_pretrained(cfg.embedding_model_name).to(cfg.device).eval()
                
                # 新しいインデックスを構築
                chunk_embs = embed_texts(cfg, chunk_data, emb_tok, emb_model)
                faiss_index = build_faiss_index(chunk_embs)
                
                # インデックスファイルを保存
                faiss.write_index(faiss_index, faiss_index_path)
                print(f"新しいFAISSインデックスを保存しました: {faiss_index_path}")
                
                last_data_file = current_file
            else:
                # キーワード検索モードの場合はFAISSインデックスは不要
                faiss_index = None
                last_data_file = current_file
                print("キーワード検索モードのため、FAISSインデックスは構築しません")
            
        except Exception as e:
            print(f"FAISSインデックス構築エラー: {e}")
            return False
    
    return True

# ==============
# 文字正規化
# ==============
def _normalize(text: str) -> str:
    # 改行削除＋NFKC＋lower
    return unicodedata.normalize("NFKC", text.replace("\n", "")).lower()


# ==========
# チャンク化
# ==========
def character_chunks(text: str, chunk_size: int, overlap: bool = False, overlap_size: int = 20) -> List[str]:
    if not text:
        return []
    chunks: List[str] = []
    step = max(1, chunk_size - overlap_size) if overlap else chunk_size
    for i in range(0, len(text), step):
        piece = text[i:i + chunk_size]
        if piece:
            chunks.append(piece)
    return chunks


def sentence_chunks(text: str, overlap: bool = False) -> List[str]:
    # 句点/？/！で文区切り（句読点は文末に含める）
    pattern = r'[^。！？!?。]+[。！？!?。]?'
    sentences = [s.strip() for s in re.findall(pattern, text) if s and s.strip()]
    if not overlap:
        return sentences
    chunks: List[str] = []
    for i, s in enumerate(sentences):
        if i == 0:
            chunks.append(s)
        else:
            chunks.append(sentences[i-1] + s)  # 直前の文を重ねる
    return chunks


def build_chunks(cfg: RAG_CONFIG, raw_text: str) -> List[str]:
    text = _normalize(raw_text)
    if cfg.chunk_mode == "char":
        return character_chunks(text, cfg.char_chunk_size, overlap=cfg.overlap, overlap_size=cfg.overlap_chars)
    elif cfg.chunk_mode == "sentence":
        return sentence_chunks(text, overlap=cfg.overlap)
    else:
        raise ValueError('chunk_mode must be "char" or "sentence"')


# ===============
# BM25トークナイズ
# ===============
def _char_bigrams(s: str) -> List[str]:
    if len(s) <= 1:
        return [s] if s else []
    return [s[i:i+2] for i in range(len(s)-1)]


def _simple_words(s: str) -> List[str]:
    return re.findall(r"[ぁ-んァ-ヶ一-龥A-Za-z0-9]+", s)


def _tokenize_ja(text: str) -> List[str]:
    # 1) Janome（原形・主要品詞）
    if _HAS_JANOME:
        t = JanomeTokenizer()
        allowed = {"名詞", "動詞", "形容詞", "副詞"}
        toks = []
        for tok in t.tokenize(text):
            pos = tok.part_of_speech.split(",")[0]
            if pos in allowed:
                base = tok.base_form if tok.base_form != "*" else tok.surface
                base = _normalize(base)
                if base:
                    toks.append(base)
        if toks:
            return toks
    # 2) 単語抽出
    toks = [_normalize(w) for w in _simple_words(text)]
    if toks:
        return toks
    # 3) バイグラムフォールバック
    grams = _char_bigrams(text)
    return grams if grams else ["_empty_"]


# ============
# 埋め込み系
# ============
@torch.no_grad()
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state  # (B, T, H)
    mask = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    summed = (token_embeddings * mask).sum(dim=1)
    counts = mask.sum(dim=1).clamp(min=1e-9)
    return summed / counts


@torch.no_grad()
def embed_texts(cfg: RAG_CONFIG,
                texts: List[str],
                tok: AutoTokenizer,
                model: AutoModel,
                batch_size: int = 32) -> np.ndarray:
    vecs = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        inputs = tok(batch, padding=True, truncation=True, return_tensors="pt").to(cfg.device)
        outputs = model(**inputs)
        emb = mean_pooling(outputs, inputs["attention_mask"])
        emb = torch.nn.functional.normalize(emb, p=2, dim=1)
        vecs.append(emb.cpu().numpy())
    return np.vstack(vecs)


# ==================
# ベクトル検索/FAISS
# ==================
def build_faiss_index(embs: np.ndarray):
    index = faiss.IndexFlatIP(embs.shape[1])
    index.add(embs)
    return index


def vector_search_top_k(cfg: RAG_CONFIG,
                        query: str,
                        chunks: List[str],
                        emb_tok: AutoTokenizer,
                        emb_model: AutoModel,
                        index,
                        k: int) -> List[Tuple[int, float]]:
    import faiss  # noqa
    with torch.no_grad():
        q_inputs = emb_tok([query], padding=True, truncation=True, return_tensors="pt").to(cfg.device)
        q_out = emb_model(**q_inputs)
        q_vec = mean_pooling(q_out, q_inputs["attention_mask"])
        q_vec = torch.nn.functional.normalize(q_vec, p=2, dim=1).cpu().numpy()
    sims, ids = index.search(q_vec, k)
    return [(int(i), float(s)) for i, s in zip(ids[0], sims[0]) if 0 <= i < len(chunks)]


# =========
# BM25検索
# =========
def keyword_search_top_k(query: str,
                         chunks: List[str],
                         k: int) -> List[Tuple[int, float]]:
    tokenized_docs = [_tokenize_ja(doc) for doc in chunks]
    tokenized_docs = [t if t else ["_empty_"] for t in tokenized_docs]
    bm25 = BM25Okapi(tokenized_docs)
    tq = _tokenize_ja(_normalize(query)) or ["_empty_"]
    scores = bm25.get_scores(tq)
    order = np.argsort(scores)[::-1][:k]
    return [(int(i), float(scores[i])) for i in order]


# =======
# リランク
# =======
@torch.no_grad()
def rerank(cfg: RAG_CONFIG,
           query: str,
           docs: List[str],
           tok: AutoTokenizer,
           model: AutoModelForSequenceClassification,
           batch_size: int = 16) -> List[int]:
    scores_all: List[float] = []
    for i in range(0, len(docs), batch_size):
        batch_docs = docs[i:i+batch_size]
        inputs = tok([query] * len(batch_docs), batch_docs,
                     padding=True, truncation=True, return_tensors="pt").to(cfg.device)
        outputs = model(**inputs)
        scores = outputs.logits.squeeze(-1).detach().cpu().numpy().tolist()
        scores_all.extend(scores)
    return np.argsort(scores_all)[::-1].tolist()


# ====================
# LLM ロード（ChangeLLM.pyと同じ方式）
# ====================
def load_clean_base_model():
    """ベースモデルを読み込み（4bit量子化対応）"""
    global llm_tok, llm
    
    try:
        model_name = "./sbintuitions/sarashina2.2-3b-instruct-v0.1"
        print(f"ベースモデルを読み込み中: {model_name}")
        
        # 4bit量子化設定（メモリ最適化）
        from transformers import BitsAndBytesConfig
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16
        )
        
        # トークナイザーを読み込み
        print("トークナイザーを読み込み中...")
        llm_tok = AutoTokenizer.from_pretrained(model_name)
        print("トークナイザーの読み込み完了")
        
        # モデルを4bit量子化で読み込み
        print("4bit量子化モデルを読み込み中...")
        llm = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=bnb_config,
            device_map="auto",
            torch_dtype=torch.float16,
            trust_remote_code=True,
            low_cpu_mem_usage=True
        )
        
        # pad_tokenが設定されていない場合の処理
        if llm_tok.pad_token_id is None and llm_tok.eos_token_id is not None:
            llm_tok.pad_token = llm_tok.eos_token
        llm.config.pad_token_id = llm_tok.pad_token_id or llm_tok.eos_token_id
        
        # 推論モードに設定
        llm.eval()
        
        print("✅ ベースモデルの読み込み完了")
        return llm_tok, llm
        
    except Exception as e:
        print(f"ベースモデルの読み込みエラー: {e}")
        return None, None

def apply_lora_adapter(base_model, adapter_path):
    """LoRAアダプターを適用（一度だけ実行）"""
    try:
        # パスの存在確認
        if not os.path.exists(adapter_path):
            print(f"LoRAアダプターパスが存在しません: {adapter_path}")
            return None
        
        from peft import PeftModel, PeftConfig
        import warnings
        
        print(f"LoRAアダプターを適用中: {adapter_path}")
        
        # LoRA設定の読み込み
        peft_config = PeftConfig.from_pretrained(adapter_path)
        print(f"LoRA設定を読み込みました: {peft_config.base_model_name_or_path}")
        
        # 警告を一時的に抑制
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=UserWarning, module="peft")
            
            # アダプターの適用
            lora_model = PeftModel.from_pretrained(
                base_model,
                adapter_path,
                is_trainable=False,  # 推論時はFalse
                torch_dtype=torch.float32
            )
        
        print("LoRAアダプターの適用完了")
        return lora_model
        
    except ImportError as e:
        print(f"PEFTライブラリがインストールされていません: {e}")
        return None
    except Exception as e:
        print(f"LoRAアダプターの適用に失敗: {e}")
        return None


# =========
# プロンプト（外出し対応）
# =========
DEFAULT_RAG_USER = (
    "与えられた情報にだけ基づいて回答してください。\n"
    "[質問]\n{question}\n\n"
    "[情報]\n{context}\n\n"
    "[回答]\n"
)

DEFAULT_FT_PROMPT = "日本語で簡潔かつ正確に回答してください。"

def _safe_format(template: str, **kwargs) -> str:
    class _D(dict):
        def __missing__(self, k): return "{"+k+"}"
    return template.format_map(_D(**kwargs))

@torch.no_grad()
def build_prompt_rag(cfg: RAG_CONFIG, question: str, retrieved_chunks: List[str], tok: AutoTokenizer) -> str:
    context = "\n---\n".join(retrieved_chunks)
    user_tmpl = cfg.rag_user_prompt_template.strip() if cfg.rag_user_prompt_template else DEFAULT_RAG_USER
    user_content = _safe_format(user_tmpl, question=question, context=context)

    if cfg.use_chat_template and hasattr(tok, "apply_chat_template"):
        messages = [
            {"role": "user", "content": user_content},
        ]
        return tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return user_content

@torch.no_grad()
def build_prompt_ft(cfg: RAG_CONFIG, question: str, tok: AutoTokenizer) -> str:
    system_tmpl = cfg.ft_system_prompt.strip() if cfg.ft_system_prompt else DEFAULT_FT_PROMPT
    
    if cfg.use_chat_template and hasattr(tok, "apply_chat_template"):
        messages = [
            {"role": "system", "content": system_tmpl},
            {"role": "user", "content": question},
        ]
        return tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return question


# ============================
# トークン制限（文末スナップ付）
# ============================
def _snap_to_sentence_boundary(text: str) -> str:
    m = re.search(r'(.+?[。．.!！?？])(?:[^。．.!！?？]*)$', text)
    return m.group(1) if m else text

def truncate_to_token_limit(tok: AutoTokenizer, text: str, limit: int) -> str:
    ids = tok(text, add_special_tokens=False).input_ids
    if len(ids) <= limit:
        return _snap_to_sentence_boundary(text)
    clipped = tok.decode(ids[:limit], skip_special_tokens=True)
    return _snap_to_sentence_boundary(clipped)


# =====
# 生成
# =====
@torch.no_grad()
def generate_with_limit(cfg: RAG_CONFIG,
                        prompt: str,
                        llm_tok: AutoTokenizer,
                        llm: AutoModelForCausalLM) -> str:
    print(f"\n=== generate_with_limit デバッグ ===")
    print(f"入力プロンプト: {prompt}")
    print(f"使用モデル型: {type(llm)}")
    print(f"モデルデバイス: {next(llm.parameters()).device if hasattr(llm, 'parameters') else 'Unknown'}")
    
    inputs = llm_tok(prompt, return_tensors="pt", truncation=True, max_length=2048).to(cfg.device)
    print(f"トークナイザー出力: input_ids shape: {inputs.input_ids.shape}")
    print(f"入力デバイス: {inputs.input_ids.device}")
    
    out = llm.generate(
        **inputs,
        max_new_tokens=cfg.answer_token_limit + cfg.answer_headroom,
        temperature=cfg.temperature,
        top_p=0.9,
        do_sample=True,
        repetition_penalty=1.05,
        pad_token_id=llm_tok.pad_token_id,
        eos_token_id=llm_tok.eos_token_id or llm_tok.pad_token_id,
    )
    
    print(f"生成出力 shape: {out.shape}")
    raw = llm_tok.decode(out[0], skip_special_tokens=True)
    print(f"生の生成結果: {raw}")
    
    text = raw.split("[回答]", 1)[-1].strip() if "[回答]" in raw else raw.strip()
    print(f"後処理後のテキスト: {text}")
    
    final_text = truncate_to_token_limit(llm_tok, text, cfg.answer_token_limit)
    print(f"最終テキスト: {final_text}")
    print("=== generate_with_limit デバッグ終了 ===\n")
    
    return final_text


# ===========
# 実行パス
# ===========
def run_pipeline(cfg: RAG_CONFIG, question: str):
    global llm_tok, llm, llm_with_lora, emb_tok, emb_model, rerank_tok, rerank_model, model_type
    
    # === デバッグ情報出力 ===
    print(f"\n=== デバッグ情報 ===")
    print(f"model_type: {model_type}")
    print(f"llm_with_lora is None: {llm_with_lora is None}")
    print(f"llm is None: {llm is None}")
    if llm_with_lora is not None:
        print(f"LoRAモデルの型: {type(llm_with_lora)}")
    if llm is not None:
        print(f"ベースモデルの型: {type(llm)}")
    
    # モデルの読み込み状態を確認
    if llm_tok is None or llm is None:
        return "エラー: 生成モデルが読み込まれていません", None
    
    # === モデル選択（ChangeLLM.pyと同じ方式） ===
    is_ft = (model_type == "finetuning" and llm_with_lora is not None)
    current_model = llm_with_lora if is_ft else llm
    print(f"モード: {'FT' if is_ft else 'ベース'}モデル")
    
    if is_ft:
        print(f"✅ LoRA適用済みモデルを使用: {type(current_model)}")
        # LoRAモデルの詳細情報を確認
        if hasattr(current_model, 'peft_config'):
            print(f"   - LoRA設定: {current_model.peft_config}")
        if hasattr(current_model, 'base_model'):
            print(f"   - ベースモデル型: {type(current_model.base_model)}")
    else:
        print(f"✅ ベースモデルを使用: {type(current_model)}")
    
    print(f"最終的に使用するモデル: {type(current_model)}")
    print(f"モデルのデバイス: {next(current_model.parameters()).device if hasattr(current_model, 'parameters') else 'Unknown'}")
    print("=== デバッグ情報終了 ===\n")
    
    # === FT モード：RAGを通さず q のみ ===
    if cfg.gen_mode == "FT":
        prompt = build_prompt_ft(cfg, question, llm_tok)
        print(f"\n=== FTモード プロンプト ===")
        print(f"プロンプト: {prompt}")
        print(f"プロンプト長: {len(prompt)}文字")
        print("=== プロンプト終了 ===\n")
        
        answer = generate_with_limit(cfg, prompt, llm_tok, current_model)
        print("\n=== 質問 ===")
        print(question)
        print("\n=== 回答 ===")
        print(answer)
        return answer, None

    # === RAG モード ===
    print(f"RAGモードで処理を開始します...")
    # 1) RAGデータの更新確認（共通のFAISSインデックスを使用）
    if not update_rag_data(cfg):
        return "エラー: RAGデータの更新に失敗しました", None
    
    # グローバル変数から取得
    global faiss_index, chunk_data
    
    # 2) 検索（5件）
    if cfg.search_mode == "vector":
        if faiss_index is None:
            return "エラー: ベクトル検索モードですがFAISSインデックスが構築されていません", None
        
        # 事前構築されたインデックスを使用
        first_stage = vector_search_top_k(cfg, question, chunk_data, emb_tok, emb_model, faiss_index, k=cfg.top_k_retrieve)
    elif cfg.search_mode == "keyword":
        first_stage = keyword_search_top_k(question, chunk_data, k=cfg.top_k_retrieve)
    else:
        raise ValueError('search_mode must be "vector" or "keyword"')

    cand_ids = [i for i, _ in first_stage]
    candidates = [chunk_data[i] for i in cand_ids]

    # 3) リランク（5→3） or そのまま3
    if cfg.rerank_mode and len(candidates) > 0:
        order = rerank(cfg, question, candidates, rerank_tok, rerank_model)
        cand_ids = [cand_ids[i] for i in order]
        candidates = [candidates[i] for i in order]

    final_ids = cand_ids[:cfg.top_k_final]
    retrieved = [chunk_data[i] for i in final_ids]

    # 4) 生成
    prompt = build_prompt_rag(cfg, question, retrieved, llm_tok)
    answer = generate_with_limit(cfg, prompt, llm_tok, current_model)

    # ===== ログ（全文表示）=====
    print(f"\n=== 検索結果（{cfg.search_mode}, Top3表示／実際はTop{cfg.top_k_retrieve}取得, "
          f"CHUNK_MODE={cfg.chunk_mode}, OVER_LAP={cfg.overlap}）===\n")
    for rank, (i, s) in enumerate(first_stage[:cfg.top_k_final], 1):
        print(f"[検索Top{rank}] id={i}, score={s:.6f}")
        print(chunk_data[i])  # 全文表示
        print("-" * 40)

    if cfg.rerank_mode:
        print("\n=== リランク後（Top3採用, 全文）===\n")
        for rank, i in enumerate(final_ids, 1):
            print(f"[RerankTop{rank}] id={i}")
            print(chunk_data[i])
            print("-" * 40)

    print("\n=== 質問 ===")
    print(question)
    print("\n=== 回答 ===")
    print(answer)
    
    # 検索結果情報を構築
    references_info = []
    for rank, i in enumerate(final_ids, 1):
        chunk_text = chunk_data[i]
        # 長すぎる場合は短縮
        if len(chunk_text) > 200:
            chunk_text = chunk_text[:200] + "..."
        references_info.append(f"**参考文献{rank}** (ID: {i})\n{chunk_text}")
    
    references_text = "\n\n".join(references_info)
    
    return answer, references_text


# ================
# Gradio UI
# ================
cfg = RAG_CONFIG()  # 基本設定（適宜書き換え）

with gr.Blocks(title="Last Chatbot", theme=gr.themes.Soft(primary_hue="blue", secondary_hue="gray")) as demo:
    gr.Markdown("## Last Chatbot")

    with gr.Row():
        # ===== 左ペイン（設定） =====
        with gr.Column(scale=1, min_width=360):
            with gr.Accordion("モデル & 実行設定", open=True):
                # チェックボックス群
                cb_no_rag   = gr.Checkbox(label="RAGなし", value=False, interactive=True)
                cb_sentence = gr.Checkbox(label="文単位チャンク分け", value=(cfg.chunk_mode=="sentence"))
                cb_overlap  = gr.Checkbox(label="オーバーラップ", value=cfg.overlap)
                cb_rerank   = gr.Checkbox(label="リランキング", value=cfg.rerank_mode)

                # 検索モード（ベクトル / キーワード）
                rd_search = gr.Radio(
                    choices=["ベクトル", "キーワード"],
                    value="ベクトル" if cfg.search_mode=="vector" else "キーワード",
                    label="検索モード",
                )

                # 数値入力（初期状態では表示）
                num_chunk = gr.Number(label="チャンクサイズ（文字数）", value=cfg.char_chunk_size, precision=0, interactive=True, visible=True)
                num_overlap = gr.Number(label="オーバーラップサイズ（文字数）", value=cfg.overlap_chars, precision=0, interactive=True, visible=cfg.overlap)

                # モデル選択（ChangeLLM.pyと同じ方式）
                model_radio = gr.Radio(
                    choices=["original", "finetuning"],
                    value="original",
                    label="モデル選択",
                    info="original：デフォルトのsarashina2.2-3b-instruct-v0.1モデルを使用、finetuning：設定したファインチューニングモデルを使用"
                )

                # プロンプト（編集可）
                with gr.Accordion("プロンプト（編集可）", open=True):
                    tb_prompt = gr.Textbox(
                        label="システムプロンプト",
                        value=DEFAULT_RAG_USER,  # 初期はデフォルトRAGプロンプト
                        lines=8,
                        max_lines=20,
                        interactive=True
                    )
                    btn_reset_prompt = gr.Button("🔄 リセット", size="sm", variant="secondary")

                # RAGデータ入力
                rag_data_accordion = gr.Accordion("RAGデータ設定", open=True)
                with rag_data_accordion:
                    # ファイルパス入力
                    tb_file_path = gr.Textbox(
                        label="ファイルパス",
                        value=cfg.text_path,
                        placeholder="例: ./data.txt",
                        interactive=True
                    )

                # 表示制御関数
                def _toggle_chunk_visibility(is_sentence: bool, is_overlap: bool):
                    # チャンクサイズの表示制御
                    chunk_vis = gr.update(visible=not is_sentence)
                    # 文単位の場合はオーバーラップサイズを非表示
                    if is_sentence:
                        overlap_vis = gr.update(visible=False)
                    else:
                        overlap_vis = gr.update(visible=is_overlap)
                    return chunk_vis, overlap_vis

                def _toggle_overlap(is_overlap: bool, is_sentence: bool):
                    # 文単位チャンク分けの場合はオーバーラップサイズを非表示
                    if is_sentence:
                        return gr.update(visible=False)
                    return gr.update(visible=is_overlap)

                def _on_no_rag_change(is_no_rag: bool):
                    # RAGなし時はチャンク関連の設定を非表示・非アクティブ化
                    if is_no_rag:
                        # チャンクサイズ関連を非表示
                        chunk_vis = gr.update(visible=False)
                        overlap_vis = gr.update(visible=False)
                        # 検索関連の設定を非表示・非アクティブ化
                        sentence_interactive = gr.update(visible=False, interactive=False)
                        overlap_interactive = gr.update(visible=False, interactive=False)
                        rerank_interactive = gr.update(visible=False, interactive=False)
                        search_visible = gr.update(visible=False)  # 検索モードを非表示
                        # プロンプトをFT用に変更
                        prompt_content = DEFAULT_FT_PROMPT
                        print(f"RAGモードをFTモードに変更: プロンプトを{DEFAULT_FT_PROMPT}に更新")
                    else:
                        # RAGモード時は通常表示・アクティブ
                        chunk_vis = gr.update(visible=not (cfg.chunk_mode=="sentence"))
                        overlap_vis = gr.update(visible=cfg.overlap)
                        sentence_interactive = gr.update(visible=True, interactive=True)
                        overlap_interactive = gr.update(visible=True, interactive=True)
                        rerank_interactive = gr.update(visible=True, interactive=True)
                        search_visible = gr.update(visible=True)  # 検索モードを表示
                        # プロンプトをRAG用に変更
                        prompt_content = DEFAULT_RAG_USER
                        print(f"RAGモードをRAGモードに変更: プロンプトを{DEFAULT_RAG_USER}に更新")
                    
                    return (chunk_vis, overlap_vis, 
                            sentence_interactive, overlap_interactive, rerank_interactive, 
                            search_visible, prompt_content)



                cb_sentence.change(_toggle_chunk_visibility, inputs=[cb_sentence, cb_overlap], outputs=[num_chunk, num_overlap])
                cb_overlap.change(_toggle_overlap, inputs=[cb_overlap, cb_sentence], outputs=num_overlap)
                # RAGモード変更時のイベントハンドラー
                def on_rag_mode_change(rag_mode):
                    if rag_mode:
                        cfg.gen_mode = "FT"
                    else:
                        cfg.gen_mode = "RAG"
                    return rag_mode
                
                # プロンプトリセット機能
                def reset_prompt_to_default(is_no_rag: bool):
                    """現在のモードに応じてデフォルトプロンプトにリセット"""
                    if is_no_rag:
                        # FTモードの場合
                        return DEFAULT_FT_PROMPT
                    else:
                        # RAGモードの場合
                        return DEFAULT_RAG_USER
                
                cb_no_rag.change(_on_no_rag_change, inputs=cb_no_rag, outputs=[
                    num_chunk, num_overlap, 
                    cb_sentence, cb_overlap, cb_rerank, rd_search, tb_prompt
                ])
                
                # リセットボタンのクリックイベント
                btn_reset_prompt.click(
                    fn=reset_prompt_to_default,
                    inputs=[cb_no_rag],
                    outputs=[tb_prompt]
                )
                
                # モデル選択の変更を監視
                def on_model_change(model_mode):
                    global model_type
                    print(f"\n=== モデル選択変更 ===")
                    print(f"変更前のmodel_type: {model_type}")
                    print(f"変更後のmodel_type: {model_mode}")
                    model_type = model_mode
                    print(f"✅ model_typeを更新しました: {model_type}")
                    print("=== モデル選択変更終了 ===\n")
                
                model_radio.change(
                    fn=on_model_change,
                    inputs=[model_radio],
                    outputs=[]
                )
                


        # ===== 右ペイン（チャット） =====
        with gr.Column(scale=2):
            chatbot = gr.Chatbot(height=520, bubble_full_width=False, show_copy_button=True)
            tb_question = gr.Textbox(label="質問", placeholder="ここに質問を入力")
            btn_send = gr.Button("送信", variant="primary")

            # 実行関数
            def _run_chat(
                q, history,
                is_no_rag, is_sentence, is_overlap, is_rerank,
                search_choice,
                chunk_size, overlap_size,
                model_mode, user_prompt, file_path
            ):
                if not q or not str(q).strip():
                    return (history or []) + [[q, "エラー: 質問を入力してください。"]], ""  # ← 送信後クリア

                # cfg を組み立て
                run_cfg = RAG_CONFIG()
                # RAGモードの判定（ChangeLLM.pyと同じ方式）
                run_cfg.gen_mode = "FT" if is_no_rag else "RAG"
                run_cfg.chunk_mode = "sentence" if is_sentence else "char"
                run_cfg.overlap = bool(is_overlap)
                # 検索モードの反映
                run_cfg.search_mode = "vector" if ("ベクトル" in search_choice) else "keyword"
                
                print(f"RAGモード設定: {'FT' if is_no_rag else 'RAG'}")
                print(f"チャンクモード: {run_cfg.chunk_mode}")
                print(f"検索モード: {run_cfg.search_mode}")

                # 数値バリデーション
                if run_cfg.chunk_mode == "char":
                    try:
                        run_cfg.char_chunk_size = int(chunk_size)
                        if run_cfg.char_chunk_size <= 0:
                            return (history or []) + [[q, "エラー: チャンクサイズは1以上で指定してください。"]], ""
                    except Exception:
                        return (history or []) + [[q, "エラー: チャンクサイズの数値が不正です。"]], ""
                try:
                    run_cfg.overlap_chars = int(overlap_size) if is_overlap else run_cfg.overlap_chars
                    if is_overlap and run_cfg.chunk_mode == "char" and run_cfg.overlap_chars >= run_cfg.char_chunk_size:
                        return (history or []) + [[q, "エラー: オーバーラップサイズはチャンクサイズ未満にしてください。"]], ""
                except Exception:
                    return (history or []) + [[q, "エラー: オーバーラップサイズの数値が不正です。"]], ""

                run_cfg.rerank_mode = bool(is_rerank)
                
                # グローバル変数のモデルタイプを更新
                global model_type
                print(f"\n=== 実行時のデバッグ情報 ===")
                print(f"変更前のmodel_type: {model_type}")
                print(f"UIから受け取ったmodel_mode: {model_mode}")
                model_type = model_mode
                print(f"✅ model_typeを更新しました: {model_type}")
                print(f"llm_with_lora is None: {llm_with_lora is None}")
                print(f"run_cfg.gen_mode: {run_cfg.gen_mode}")
                
                # モデル選択の状態を確認
                is_ft = (model_type == "finetuning" and llm_with_lora is not None)
                print(f"使用予定モデル: {'LoRA適用済み' if is_ft else 'ベース'}モデル")
                print("=== 実行時デバッグ情報終了 ===\n")
                
                # プロンプト適用：現在のモードに応じて片方に入れる（ChangeLLM.pyと同じ方式）
                if run_cfg.gen_mode == "FT":
                    run_cfg.ft_system_prompt = user_prompt or ""
                    run_cfg.rag_user_prompt_template = ""  # RAG側はデフォルトに戻す
                    print(f"FTモード用プロンプトを適用: {run_cfg.ft_system_prompt}")
                else:
                    run_cfg.rag_user_prompt_template = user_prompt or ""
                    run_cfg.ft_system_prompt = ""   # FT側はデフォルトに戻す
                    print(f"RAGモード用プロンプトを適用: {run_cfg.rag_user_prompt_template}")

                # RAGなし時の必須チェック（ChangeLLM.pyと同じ方式）
                if run_cfg.gen_mode == "FT" and model_type == "finetuning" and llm_with_lora is None:
                    print(f"⚠️ エラー: RAGなし時はLoRAアダプターが適用されている必要があります")
                    print(f"   - gen_mode: {run_cfg.gen_mode}")
                    print(f"   - model_type: {model_type}")
                    print(f"   - llm_with_lora is None: {llm_with_lora is None}")
                    return (history or []) + [[q, "エラー: RAGなし時はLoRAアダプターが適用されている必要があります。"]], ""
                
                # RAGモード時のファイルパス更新
                if run_cfg.gen_mode == "RAG" and file_path and file_path.strip():
                    run_cfg.text_path = file_path.strip()
                    print(f"RAGデータファイルパスを更新: {run_cfg.text_path}")

                # 実行
                print(f"\n=== 実行開始 ===")
                print(f"最終的な設定:")
                print(f"  - gen_mode: {run_cfg.gen_mode}")
                print(f"  - model_type: {model_type}")
                print(f"  - chunk_mode: {run_cfg.chunk_mode}")
                print(f"  - search_mode: {run_cfg.search_mode}")
                print("=== 実行開始終了 ===\n")
                
                try:
                    result = run_pipeline(run_cfg, q)
                    if isinstance(result, tuple):
                        ans, references = result
                    else:
                        ans, references = result, None
                except Exception as e:
                    ans = f"実行時エラー: {e}"
                    references = None

                # RAGモードの時は参考文献情報も含める（ChangeLLM.pyと同じ方式）
                if run_cfg.gen_mode == "RAG" and references:
                    full_answer = f"{ans}\n\n---\n**参考文献**\n{references}"
                    print(f"RAGモード: 参考文献情報を含めて回答を表示")
                else:
                    full_answer = ans
                    print(f"FTモード: 参考文献なしで回答を表示")

                # ← 送信後は質問欄をクリアするため "" を返す
                return (history or []) + [[q, full_answer]], ""

            # 出力を chatbot と tb_question（空文字でクリア）に
            btn_send.click(
                _run_chat,
                inputs=[
                    tb_question, chatbot,
                    cb_no_rag, cb_sentence, cb_overlap, cb_rerank,
                    rd_search,
                    num_chunk, num_overlap,
                    model_radio, tb_prompt, tb_file_path
                ],
                outputs=[chatbot, tb_question]
            )

            # Enterでも送信して同様にクリア
            tb_question.submit(
                _run_chat,
                inputs=[
                    tb_question, chatbot,
                    cb_no_rag, cb_sentence, cb_overlap, cb_rerank,
                    rd_search,
                    num_chunk, num_overlap,
                    model_radio, tb_prompt, tb_file_path
                ],
                outputs=[chatbot, tb_question]
            )

if __name__ == "__main__":
    print("=== チャットボット起動 ===")
    
    # 1. 生成モデル（sarashina）を読み込み
    print("1/3: 生成モデル（sarashina）を読み込み中")
    print("   - ベースモデルを読み込み中")
    llm_tok, llm = load_clean_base_model()
    if llm_tok is None or llm is None:
        print("エラー: ベースモデルの読み込みに失敗しました")
        exit(1)
    print("✅ ベースモデルの読み込み完了")
    
    # 2. 埋め込みモデル（bge-m3）を読み込み
    print("2/3: 埋め込みモデルを読み込み中")
    try:
        emb_tok = AutoTokenizer.from_pretrained("./bge-m3")
        emb_model = AutoModel.from_pretrained("./bge-m3").to(torch.device("cuda" if torch.cuda.is_available() else "cpu")).eval()
        print("✅ 埋め込みモデルの読み込み完了")
    except Exception as e:
        print(f"エラー: 埋め込みモデルの読み込みに失敗しました: {e}")
        exit(1)
    
    # 3. リランクモデルを読み込み
    print("3/3: リランクモデルを読み込み中")
    try:
        rerank_tok = AutoTokenizer.from_pretrained("./japanese-bge-reranker-v2-m3-v1")
        rerank_model = AutoModelForSequenceClassification.from_pretrained("./japanese-bge-reranker-v2-m3-v1").to(torch.device("cuda" if torch.cuda.is_available() else "cpu")).eval()
        print("✅ リランクモデルの読み込み完了")
    except Exception as e:
        print(f"エラー: リランクモデルの読み込みに失敗しました: {e}")
        exit(1)
    
    print("🎉 すべてのモデルの読み込みが完了しました！")
    
    # 4. LoRAアダプターを事前に適用
    print("4/4: LoRAアダプターを事前適用中...")
    print(f"LoRAアダプターパス: {DEFAULT_LORA_ADAPTER_PATH}")
    print(f"パスの存在確認: {os.path.exists(DEFAULT_LORA_ADAPTER_PATH)}")
    
    if os.path.exists(DEFAULT_LORA_ADAPTER_PATH):
        try:
            print(f"LoRAアダプターを適用中...")
            llm_with_lora = apply_lora_adapter(llm, DEFAULT_LORA_ADAPTER_PATH)
            if llm_with_lora is not None:
                print("✅ LoRAアダプターの事前適用完了")
                print(f"   - ベースモデル: {cfg.llm_model_name}")
                print(f"   - LoRAアダプター: {DEFAULT_LORA_ADAPTER_PATH}")
                print(f"   - LoRAモデルの型: {type(llm_with_lora)}")
                
                # LoRAモデルの詳細情報を確認
                if hasattr(llm_with_lora, 'peft_config'):
                    print(f"   - LoRA設定: {llm_with_lora.peft_config}")
                if hasattr(llm_with_lora, 'base_model'):
                    print(f"   - ベースモデル型: {type(llm_with_lora.base_model)}")
                
                # モデルのパラメータ数を確認
                try:
                    total_params = sum(p.numel() for p in llm_with_lora.parameters())
                    trainable_params = sum(p.numel() for p in llm_with_lora.parameters() if p.requires_grad)
                    print(f"   - 総パラメータ数: {total_params:,}")
                    print(f"   - 学習可能パラメータ数: {trainable_params:,}")
                except Exception as e:
                    print(f"   - パラメータ数確認エラー: {e}")
                
            else:
                print("⚠️ LoRAアダプターの適用に失敗、ベースモデルのみで続行します")
                llm_with_lora = None
        except Exception as e:
            print(f"警告: LoRAアダプターの事前適用に失敗しました: {e}")
            print("オリジナルモデルのみで続行します")
            llm_with_lora = None
    else:
        print("LoRAアダプターパスが存在しないため、オリジナルモデルのみで続行します")
        llm_with_lora = None
    
    # 最終的なモデル状態を確認
    print(f"\n=== 最終的なモデル状態 ===")
    print(f"llm (ベースモデル): {type(llm) if llm is not None else 'None'}")
    print(f"llm_with_lora (LoRAモデル): {type(llm_with_lora) if llm_with_lora is not None else 'None'}")
    print(f"model_type: {model_type}")
    print("=== モデル状態確認終了 ===\n")
    
    print("🎉 すべてのモデルの読み込みが完了しました！")
    print("UIを起動しています...")
    
    port = find_available_port(7890)
    if port is None:
        print("エラー: 利用可能なポートが見つかりませんでした。")
        exit(1)
    
    print(f"ポート {port} で起動します...")
    
    try:
        demo.launch(
            inbrowser=True,
            server_name="0.0.0.0",
            server_port=port,
            share=False,
            show_error=True
        )
    except OSError as e:
        if "Address already in use" in str(e):
            print(f"ポート {port} が使用中です。別のポートを試します...")
            # 別のポートで再試行
            port = find_available_port(port + 1)
            if port is None:
                print("エラー: 利用可能なポートが見つかりませんでした。")
                exit(1)
            
            print(f"ポート {port} で再起動します...")
            demo.launch(
                inbrowser=True,
                server_name="0.0.0.0",
                server_port=port,
                share=False,
                show_error=True
            )
        else:
            print(f"起動中にエラーが発生しました: {e}")
            raise

D:\temp\ipykernel_24716\2245983033.py:887: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=520, bubble_full_width=False, show_copy_button=True)
D:\temp\ipykernel_24716\2245983033.py:887: DeprecationWarning: The 'bubble_full_width' parameter is deprecated and will be removed in a future version. This parameter no longer has any effect.
  chatbot = gr.Chatbot(height=520, bubble_full_width=False, show_copy_button=True)


=== チャットボット起動 ===
1/3: 生成モデル（sarashina）を読み込み中
   - ベースモデルを読み込み中
ベースモデルを読み込み中: ./sbintuitions/sarashina2.2-3b-instruct-v0.1
トークナイザーを読み込み中...
トークナイザーの読み込み完了
4bit量子化モデルを読み込み中...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ ベースモデルの読み込み完了
✅ ベースモデルの読み込み完了
2/3: 埋め込みモデルを読み込み中
✅ 埋め込みモデルの読み込み完了
3/3: リランクモデルを読み込み中
✅ リランクモデルの読み込み完了
🎉 すべてのモデルの読み込みが完了しました！
4/4: LoRAアダプターを事前適用中...
LoRAアダプターパス: ./output/checkpoint-160
パスの存在確認: True
LoRAアダプターを適用中...
LoRAアダプターを適用中: ./output/checkpoint-160
LoRA設定を読み込みました: sbintuitions/sarashina2.2-3b-instruct-v0.1
LoRAアダプターの適用完了
✅ LoRAアダプターの事前適用完了
   - ベースモデル: ./sbintuitions/sarashina2.2-3b-instruct-v0.1
   - LoRAアダプター: ./output/checkpoint-160
   - LoRAモデルの型: <class 'peft.peft_model.PeftModelForCausalLM'>
   - LoRA設定: {'default': LoraConfig(task_type='CAUSAL_LM', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='sbintuitions/sarashina2.2-3b-instruct-v0.1', revision=None, inference_mode=True, r=128, target_modules={'q_proj', 'up_proj', 'down_proj', 'o_proj', 'gate_proj', 'v_proj', 'k_proj'}, exclude_modules=None, lora_alpha=128, lora_dropout=0.05, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_tr


=== モデル選択変更 ===
変更前のmodel_type: original
変更後のmodel_type: finetuning
✅ model_typeを更新しました: finetuning
=== モデル選択変更終了 ===


=== モデル選択変更 ===
変更前のmodel_type: finetuning
変更後のmodel_type: original
✅ model_typeを更新しました: original
=== モデル選択変更終了 ===

RAGモードをFTモードに変更: プロンプトを日本語で簡潔かつ正確に回答してください。に更新

=== モデル選択変更 ===
変更前のmodel_type: original
変更後のmodel_type: finetuning
✅ model_typeを更新しました: finetuning
=== モデル選択変更終了 ===


=== モデル選択変更 ===
変更前のmodel_type: finetuning
変更後のmodel_type: original
✅ model_typeを更新しました: original
=== モデル選択変更終了 ===

RAGモード設定: FT
チャンクモード: char
検索モード: vector

=== 実行時のデバッグ情報 ===
変更前のmodel_type: original
UIから受け取ったmodel_mode: original
✅ model_typeを更新しました: original
llm_with_lora is None: False
run_cfg.gen_mode: FT
使用予定モデル: ベースモデル
=== 実行時デバッグ情報終了 ===

FTモード用プロンプトを適用: あなたは大谷翔平について知り尽くしているエキスパートです。
日本語で簡潔かつ正確に回答してください。

=== 実行開始 ===
最終的な設定:
  - gen_mode: FT
  - model_type: original
  - chunk_mode: char
  - search_mode: vector
=== 実行開始終了 ===


=== デバッグ情報 ===
model_type: original
llm_with_lora i